In [ ]:
#Note: Code inspired by https://sdss-marvin.readthedocs.io/en/latest/tools/bpt.html
from matplotlib import pyplot as plt
from matplotlib import colors
import numpy as np
from marvin.tools.maps import Maps
import marvin.utils.plot.map as mapplot
from marvin.tools.image import Image

In [ ]:
def get_masked(maps, emline):
    """Function to get masked data arrays without bad snr spaxels

    Parameters:
    maps - Marvin Maps object of the galaxy
    emline - String of desired quantity (e.g. 'emline_gflux_ha_6564')

    Returns:
    Data array with masked values if the spaxel has an snr < 5
    """    
    data_array = maps[emline]
    data_array_masked = data_array.masked

    #Masks all spaxels that don't reach the cutoff SNR
    data_array_masked.mask |= maps.bin_snr.value < 5
    
    return data_array_masked

In [ ]:
def get_flattened(x_s, y_s):
    """Function to return the 1d lists of x and y values given
    their data arrays

    Parameters:
    x_s - Data array of x values
    y_s - Data array of y values

    Returns:
    Tuple containing list of x values and the corresponding list of y
    values for all pairs in which neither is masked
    """  
    flat_x = []
    flat_y = []
    flat_vals = []
    
    for i in range(len(x_s)):
        for j in range(len(y_s)):
            if not (np.ma.is_masked(x_s[i][j]) or np.ma.is_masked(y_s[i][j])):
                flat_x.append(x_s[i][j])
                flat_y.append(y_s[i][j])
                flat_vals.append(value[i][j])          
                
    return flat_x, flat_y, flat_vals

In [ ]:
#Specify galaxy with MaNGAid
gal_id = '52-5'

maps = Maps(mangaid=gal_id)

nii = get_masked(maps, 'emline_gflux_nii_6585')
ha = get_masked(maps, 'emline_gflux_ha_6564')
ewha = get_masked(maps, 'emline_sew_ha_6564')
ewnii = get_masked(maps, 'emline_sew_nii_6585')
log_nii_ha = np.log10(nii/ha)

nocov = maps['emline_sew_ha_6564'].pixmask.get_mask('NOCOV')

In [ ]:
#WHAN categories
psf = (log_nii_ha < -0.4) & (ewha > 3)
sagn = (log_nii_ha > -0.4) & (ewha > 6)
wagn = (log_nii_ha > -0.4) & ((ewha > 3) & (ewha < 6))
rg = ewha < 3
pg = (ewha < 0.5) & (ewnii < 0.5)

In [ ]:
#Array for WHAN categories
value = ewha.copy()
value[psf] = 1   #Pure star-forming
value[sagn] = 2  #Strong AGN
value[wagn] = 3  #Weak AGN
value[rg] = 4    #Retired AGN
value[pg] = 5    #Passive

for i in range(len(value)):
    for j in range(len(value)):
        if value[i][j] != 1 and value[i][j] != 2 and value[i][j] != 3 and value[i][j] != 4 and value[i][j] != 5:
            value[i][j] = np.ma.masked 

#Set ivar to 0 when SNR < 5
low_snr = maps.bin_snr.value < 5
ivar = ewha.copy()
ivar[low_snr] = 0

In [ ]:
#Plot WHAN
cmap = colors.ListedColormap(['red', 'blue', 'goldenrod', 'green', 'purple'])

fig, ax, cb = mapplot.plot(value=value, ivar=ivar, mask=nocov, cmap=cmap, use_masks='NOCOV', return_cb=True, cbrange=(0.5, 5.5),
                           title='WHAN Diagram {}'.format(gal_id))
fig.set_size_inches(9, 6)

cb.set_ticks([1, 2, 3, 4, 5])
cb.set_ticklabels(['PSF', 'SAGN', 'WAGN', 'RG', 'PG'])

plt.show()

In [ ]:
#Plot snr
snr = maps.bin_snr.value
snr_max = maps.bin_snr.value.max()

fig, ax, cb = mapplot.plot(value=snr, ivar=ivar, mask=nocov, use_masks='NOCOV', return_cb=True, cbrange=(snr_max, 0),
                           title='Signal to Noise Diagram {}'.format(gal_id))
fig.set_size_inches(9, 6)
plt.show()

In [ ]:
#BPT Categories
oiii = get_masked(maps, 'emline_gflux_oiii_5008')
nii = get_masked(maps, 'emline_gflux_nii_6585')
ha = get_masked(maps, 'emline_gflux_ha_6564')
hb = get_masked(maps, 'emline_gflux_hb_4862')
oi = get_masked(maps, 'emline_gflux_oi_6302')

sii_6718 = get_masked(maps, 'emline_gflux_sii_6718')
sii_6732 = get_masked(maps, 'emline_gflux_sii_6732')
sii = sii_6718 + sii_6732

log_oiii_hb = np.log10(oiii/hb)
log_nii_ha = np.log10(nii/ha)
log_sii_ha = np.log10(sii/ha)
log_oi_ha = np.log10(oi/ha)

sf = (log_oiii_hb < 0.61/(log_nii_ha - 0.05) + 1.30) & \
     (log_nii_ha < 0.05) & \
     (log_oiii_hb < 0.72/(log_sii_ha - 0.32) + 1.30) & \
     (log_sii_ha < 0.32) & \
     (log_oiii_hb < 0.73/(log_oi_ha + 0.59) + 1.33) & \
     (log_oi_ha < -0.59)

comp = (0.61/(log_nii_ha - 0.05) + 1.30 < log_oiii_hb) & \
       (log_nii_ha < 0.05) & \
       (0.61/(log_nii_ha - 0.47) + 1.19 > log_oiii_hb) & \
       (log_nii_ha < 0.47) & \
       (log_oiii_hb < 0.72/(log_sii_ha - 0.32) + 1.30) & \
       (log_sii_ha < 0.32) & \
       (log_oiii_hb < 0.73/(log_oi_ha + 0.59) + 1.33) & \
       (log_oi_ha < -0.59)

seyfert = ((0.61/(log_nii_ha - 0.47) + 1.19 < log_oiii_hb) |
          (log_nii_ha > 0.47)) & \
          ((0.72/(log_sii_ha - 0.32) + 1.30 < log_oiii_hb) |
          (log_sii_ha > 0.32)) & \
          ((0.73/(log_oi_ha + 0.59) + 1.33 < log_oiii_hb) | \
          (log_oi_ha > -0.59)) & \
          (1.89*log_sii_ha + 0.76 < log_oiii_hb) & \
          (1.18*log_oi_ha + 1.30 < log_oiii_hb)

liner = ((0.61/(log_nii_ha - 0.47) + 1.19 < log_oiii_hb) |
        (log_nii_ha > 0.47)) & \
        ((0.72/(log_sii_ha - 0.32) + 1.30 < log_oiii_hb) |
        (log_sii_ha > 0.32)) & \
        ((0.73/(log_oi_ha + 0.59) + 1.33 < log_oiii_hb) | \
        (log_oi_ha > -0.59)) & \
        (log_oiii_hb < 1.89*log_sii_ha + 0.76) & \
        (log_oiii_hb < 1.18*log_oi_ha + 1.30)

ambig = sf.copy()

for i in range(len(ambig)):
    for j in range(len(ambig)):
        if not (np.ma.is_masked(sf[i][j]) and np.ma.is_masked(comp[i][j]) and np.ma.is_masked(seyfert[i][j]) and np.ma.is_masked(liner[i][j])):
            if not (sf[i][j] or comp[i][j] or seyfert[i][j] or liner[i][j]):
                ambig[i][j] = True
            else:
                ambig[i][j] = False
        else:
            ambig[i][j] = np.ma.masked

In [ ]:
#Array for BPT categories
value = ha.copy()
value[sf] = 1      #Star forming
value[seyfert] = 2 #SEYFERT
value[liner] = 3   #Liner
value[comp] = 4    #Composite
value[ambig] = 5   #Ambiguous

for i in range(len(value)):
    for j in range(len(value)):
        if value[i][j] != 1 and value[i][j] != 2 and value[i][j] != 3 and value[i][j] != 4 and value[i][j] != 5:
            value[i][j] = np.ma.masked

In [ ]:
#Plot BPT spaxels
cmap = colors.ListedColormap(['red', 'blue', 'goldenrod', 'chartreuse', 'grey'])

fig, ax, cb = mapplot.plot(value=value, ivar=ivar, mask=nocov, cmap=cmap, use_masks='NOCOV', return_cb=True, cbrange=(0.5, 5.5),
                           title='BPT Spaxel Diagram {}'.format(gal_id))
fig.set_size_inches(9, 6)
cb.set_ticks([1, 2, 3, 4, 5])
cb.set_ticklabels(['SF', 'SEYFERT', 'LINER', 'Comp', 'Ambig'])

plt.show()

In [ ]:
#Plot NII BPT graph
flat_log_nii_ha, flat_log_oiii_hb, vals = get_flattened(log_nii_ha, log_oiii_hb)
gals = ['Star Forming', 'SEYFERT', 'LINER', 'Composite', 'Ambiguous']
cols = ['red', 'blue', 'goldenrod', 'chartreuse', 'grey']

plt.figure(figsize=(15, 9))
for i in range(len(gals)):
    x_s = []
    y_s = []
    
    for j in range(len(vals)):
        if vals[j] == (i + 1):
            x_s.append(flat_log_nii_ha[j])
            y_s.append(flat_log_oiii_hb[j])
            
    plt.scatter(x_s, y_s, c=cols[i])
        
plt.xlim(-1, 0.75)
plt.ylim(-1.5, 1.5)
plt.xlabel(r'$log([NII]/H\alpha)$', fontsize=14)
plt.ylabel(r'$log([OIII]/H\beta)$', fontsize=14)
x = np.linspace(-1, 0.47)
x2 = np.linspace(-2, 0.05)
y = 0.61/(x - 0.47) + 1.19
y2 = 0.61/(x2 - 0.05) + 1.30

plt.legend(gals, fontsize=14)
plt.plot(x, y, color='black', linewidth=4)
plt.plot(x2, y2, color='black', linewidth=4)

In [ ]:
#Plot SII BPT graph
flat_log_sii_ha, flat_log_oiii_hb, vals = get_flattened(log_sii_ha, log_oiii_hb)
gals = ['Star Forming', 'SEYFERT', 'LINER', 'Composite', 'Ambiguous']
cols = ['red', 'blue', 'goldenrod', 'chartreuse', 'grey']

plt.figure(figsize=(15, 9))
for i in range(len(gals)):
    x_s = []
    y_s = []
    
    for j in range(len(vals)):
        if vals[j] == (i + 1):
            x_s.append(flat_log_sii_ha[j])
            y_s.append(flat_log_oiii_hb[j])
            
    plt.scatter(x_s, y_s, c=cols[i])

plt.xlim(-1.0, 0.5)
plt.ylim(-1.5, 1.5)
plt.xlabel(r'$log([SII]/H\alpha)$', fontsize=14)
plt.ylabel(r'$log([OIII]/H\beta)$', fontsize=14)
x = np.linspace(-1.0, 0.32)
x2 = np.linspace(-0.3146, 0.5)
y = 0.72/(x - 0.32) + 1.30
y2 = 1.89*x2 + 0.76

plt.legend(gals, fontsize=14)
plt.plot(x, y, color='black', linewidth=4)
plt.plot(x2, y2, color='black', linewidth=4)

In [ ]:
#Plot OI BPT graph
flat_log_oi_ha, flat_log_oiii_hb, vals = get_flattened(log_oi_ha, log_oiii_hb)
gals = ['Star Forming', 'SEYFERT', 'LINER', 'Composite', 'Ambiguous']
cols = ['red', 'blue', 'goldenrod', 'chartreuse', 'grey']

plt.figure(figsize=(15, 9))
for i in range(len(gals)):
    x_s = []
    y_s = []
    
    for j in range(len(vals)):
        if vals[j] == (i + 1):
            x_s.append(flat_log_oi_ha[j])
            y_s.append(flat_log_oiii_hb[j])
            
    plt.scatter(x_s, y_s, c=cols[i])

plt.xlim(-2.5, 0.5)
plt.ylim(-1.5, 1.5)
plt.xlabel(r'$log([OI]/H\alpha)$', fontsize=14)
plt.ylabel(r'$log([OIII]/H\beta)$', fontsize=14)
x = np.linspace(-2.5, -0.59)
x2 = np.linspace(-1.1269, 0.5)
y = 0.73/(x + 0.59) + 1.33
y2 = 1.18*x2 + 1.3

plt.legend(gals, fontsize=14)
plt.plot(x, y, color='black', linewidth=4)
plt.plot(x2, y2, color='black', linewidth=4)

In [ ]:
#Show optical image
im = Image(gal_id)
im.get_new_cutout(100, 100, scale=0.12)
ax = im.plot()